In [1]:
import math
import copy
from task import Task
from caseLoader import CaseLoader
from taskType import TaskType

In [2]:
MT = 10 # microtick

In [3]:
loader = CaseLoader()
cases = loader.load_test_case("inf_30_30", 0)
cases = [c for c in cases if c.type == TaskType.TIME]

Searching for test cases at:
./test_cases/inf_30_30
Loaded case_group: inf_30_30 - case: 0 test case(s).


In [4]:
def get_ready(task_set, cycle, ready_list):
    for task in task_set:
        if cycle % task.period == 0:
            new_job = copy.deepcopy(task)
            new_job.deadline = new_job.deadline + cycle
            new_job.release_time = cycle
            ready_list.append(new_job)
    
    # sort on deadline
    # https://www.geeksforgeeks.org/sorting-objects-of-user-defined-class-in-python/
    ready_list = sorted(ready_list, key=lambda t: t.deadline) 
    
    return sorted(ready_list, key=lambda t: t.deadline)
        

# ts is set of TT tasks
def edf(ts):
    T = math.lcm(2000,3000,4000) # least common multiple of TT task periods. 2000 mt = 
    S = [] # schedule will be hyperperiod long. 12 000 microticks == 120 000 microsecs == 120 ms
    ready_list = []
    wcrts = {} # worst case response times

    for t in range(0, T):
        ready_list = get_ready(ts, t, ready_list)
        
        for task in ready_list:
            if task.duration > 0 and task.deadline <= t:
                return []
        
            # job done check response time gt wcrt and remove from ready list
            if task.duration == 0 and task.deadline >= t:
                response_time = t - task.release_time
                
                if task.name not in wcrts or response_time >= wcrts[task.name]:
                    wcrts[task.name] = response_time
                
                ready_list.remove(task)
                
            
        if ready_list == []:
            S.append("IDLE")
            continue
        else:
            # EDF get next job to execute 
            S.append(ready_list[0].name)
            ready_list[0].duration = ready_list[0].duration - 1
    
    if ready_list != []:
        return [], []
    
    return S, wcrts

In [5]:
s, wcrts = edf(cases)

In [6]:
print(wcrts)

{'tTT0': 16, 'tTT1': 33, 'tTT4': 39, 'tTT5': 42, 'tTT6': 63, 'tTT11': 81, 'tTT14': 85, 'tTT18': 113, 'tTT21': 125, 'tTT24': 156, 'tTT26': 162, 'tTT27': 171, 'tTT7': 210, 'tTT8': 235, 'tTT9': 286, 'tTT10': 322, 'tTT17': 380, 'tTT19': 439, 'tTT20': 444, 'tTT23': 592, 'tTT25': 594, 'tTT28': 674, 'tTT2': 703, 'tTT3': 704, 'tTT12': 835, 'tTT13': 842, 'tTT15': 859, 'tTT16': 874, 'tTT22': 880, 'tTT29': 884}
